In [ ]:
!pip install snakeviz

# cProfile metrics
- __ncalls:__ Shows the number of calls made
- __tottime:__ Total time taken by the given function. Note that the time made in calls to sub-functions are excluded.
- __percall:__ Total time / No of calls. ( remainder is left out )
- __cumtime:__ Unlike tottime, this includes time spent in this and all subfunctions that the higher-level function calls. It is most useful and is accurate for recursive functions.
- The percall following cumtime is calculated as the quotient of cumtime divided by primitive calls. The primitive calls include all the calls that were not included through recursion.

In [50]:
import os
import sys
import pandas as pd
import cProfile
import pstats
import time

from pyjedai.datamodel import Data
data = Data(
    dataset_1=pd.read_csv("./data/D2/abt.csv", sep='|', engine='python', na_filter=False).astype(str),
    attributes_1=['id','name','description'],
    id_column_name_1='id',
    dataset_2=pd.read_csv("./data/D2/buy.csv", sep='|', engine='python', na_filter=False).astype(str),
    attributes_2=['id','name','description'],
    id_column_name_2='id',
    ground_truth=pd.read_csv("./data/D2/gt.csv", sep='|', engine='python'),
)
data.process()
from pyjedai.workflow import WorkFlow, compare_workflows
from pyjedai.block_building import StandardBlocking, QGramsBlocking, ExtendedQGramsBlocking, SuffixArraysBlocking, ExtendedSuffixArraysBlocking
from pyjedai.block_cleaning import BlockFiltering, BlockPurging
from pyjedai.comparison_cleaning import WeightedEdgePruning, WeightedNodePruning, CardinalityEdgePruning, CardinalityNodePruning, BLAST, ReciprocalCardinalityNodePruning, ReciprocalWeightedNodePruning, ComparisonPropagation
from pyjedai.matching import EntityMatching
from pyjedai.clustering import ConnectedComponentsClustering
from pyjedai.block_building import (
    StandardBlocking,
    QGramsBlocking,
    ExtendedQGramsBlocking,
    SuffixArraysBlocking,
    ExtendedSuffixArraysBlocking,
)

# %reload_ext snakeviz
# %snakeviz 

from pyjedai.vector_based_blocking import EmbeddingsNNBlockBuilding

qgb = StandardBlocking()

t1 = time.process_time()
%reload_ext snakeviz
%snakeviz blocks = qgb.build_blocks(data, attributes_1=['name'])
t2 = time.process_time() - t1
print(t2)

# bf = BlockFiltering(ratio=0.8)
# filtered_blocks = bf.process(blocks, data, tqdm_disable=False)
# cbbp = BlockPurging()
# cleaned_blocks = cbbp.process(filtered_blocks, data, tqdm_disable=False)
# wep = CardinalityEdgePruning(weighting_scheme='X2')
# candidate_pairs_blocks = wep.process(filtered_blocks, data, tqdm_disable=True)
# EM = EntityMatching(
#     metric='sorensen_dice',
#     similarity_threshold=0.5,
#     attributes = ['description', 'name']
# )

# pairs_graph = EM.predict(candidate_pairs_blocks, data, tqdm_disable=True)
# ccc = ConnectedComponentsClustering()
# clusters = ccc.process(pairs_graph)

Standard Blocking:   0%|          | 0/2152 [00:00<?, ?it/s]

 
*** Profile stats marshalled to file 'C:\\Users\\nikol\\AppData\\Local\\Temp\\tmpd2zub12i'. 
Embedding SnakeViz in this document...


0.25


In [51]:
data.dataset_1.columns.values.tolist()

['id', 'name', 'description', 'price']

In [54]:
df = pd.DataFrame()
df.columns.values.tolist()

[]